In [10]:
% pip install faiss-cpu

UsageError: Line magic function `%` not found.


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Loaders  
To use data with an LLM, documents must first be loaded into a vector database. 
The first step is to load them into memory via a loader

In [2]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    "./FAQ", glob="**/*.txt", loader_cls=TextLoader, show_progress=True
)
docs = loader.load()

100%|██████████| 3/3 [00:00<00:00, 2949.58it/s]


## Text splitter
Texts are not loaded 1:1 into the database, but in pieces, so called "chunks". You can define the chunk size and the overlap between the chunks.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=75,
)

documents = text_splitter.split_documents(docs)
documents[0]

Document(page_content='Q: What are the ingredients in your gluten-free options?', metadata={'source': 'FAQ/Health.txt'})

## Embeddings
Texts are not stored as text in the database, but as vector representations.
Embeddings are a type of word representation that represents the semantic meaning of words in a vector space.

In [4]:
from langchain.embeddings import HuggingFaceHubEmbeddings

repo_id = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceHubEmbeddings(
    repo_id=repo_id,
    task="feature-extraction",
)

/Users/atorres/.local/share/virtualenvs/langchain-kv9OLnAB-python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


## Loading Vectors into VectorDB (FAISS)
As created by OpenAIEmbeddings vectors can now be stored in the database. The DB can be stored as .pkl file

In [5]:
from langchain.vectorstores.faiss import FAISS
import pickle

vectorstore = FAISS.from_documents(documents, embeddings)

with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

## Loading the database
Before using the database, it must of course be loaded again.

In [6]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

## Prompts
With an LLM you have the possibility to give it an identity before a conversation or to define how question and answer should look like.

In [10]:
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant. Answer only the infomation that is asked for.

{context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

## Chains
With chain classes you can easily influence the behavior of the LLM

In [11]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_length":256, "max_new_tokens":100})

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
)

query = "When does the restaurant open?"
qa.run(query)

'\n\nA: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.'

## Memory
In the example just shown, each request stands alone. A great strength of an LLM, however, is that it can take the entire chat history into account when responding. For this, however, a chat history must be built up from the different questions and answers. With different memory classes this is very easy in Langchain.

In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True, output_key="answer"
)

## Use Memory in Chains
The memory class can now easily be used in a chain. This is recognizable, for example, by the fact that when one speaks of "it", the bot understands the rabbit in this context.

In [17]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm= HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_length":256, "max_new_tokens":100}),
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs={"prompt": PROMPT},
)


query = "Do you offer vegan food?"
qa({"question": query})
qa({"question": "How much does that type of food cost?"})

{'question': 'How much does that type of food cost?',
 'chat_history': [HumanMessage(content='Do you offer vegan food?', additional_kwargs={}, example=False),
  AIMessage(content=' Yes, we have a range of vegan dishes on our menu. Please let our staff know about any dietary restrictions you have when you order.', additional_kwargs={}, example=False),
  HumanMessage(content='How much does it cost?', additional_kwargs={}, example=False),
  AIMessage(content=' "Yes, we have a range of dishes to cater to vegetarians and vegans. Please let our staff know about any dietary restrictions you have when you order."', additional_kwargs={}, example=False),
  HumanMessage(content='Do you offer vegan food?', additional_kwargs={}, example=False),
  AIMessage(content=' \nWe offer a range of vegan dishes, including a Vegan Burger made from plant-based ingredients, a Vegan Pad Thai, and a Vegan Mushroom Risotto. Please let our staff know if you have any specific dietary requirements.', additional_kwargs